# Imports

In [1]:
#import the necessary libraries for inital data analysis
import pandas as pd

# Data Read

In [36]:
#reading the data file in a dataframe, adding low_memory=False to remove warning while reading the data
df = pd.read_csv("loan.csv",low_memory=False)

# Data understanding

In [21]:
# Just by eyeballing the data we can notice some huge data quality issues like
# Many columns invalid values like NA
# some columns contains redundant values like 0,1,INDIVIDUAL,n or more
# some columns have many missing values like columns "next_pymnt_d"
# some columns have Empty values also 

In [28]:
# Duplication Check
df1 = df.drop(["id","member_id"],axis=1)
len(df1.drop_duplicates())
# we found out that there are no duplicate rows in the updated dataframe after removing the unqiue id's from the data

39717

# Data Cleaning

In [37]:
#check shape of your dataframe i.e. how many rows and columns we have in total
df.shape

(39717, 111)

In [ ]:
# Filter out unncessary columns that do not pertain with our use case

In [38]:
df = df.drop(['collections_12_mths_ex_med','mths_since_last_major_derog','policy_code','application_type',
               'annual_inc_joint','dti_joint','verification_status_joint','acc_now_delinq','tot_coll_amt',
               'tot_cur_bal','open_acc_6m','open_il_6m','open_il_12m','open_il_24m','mths_since_rcnt_il',
               'total_bal_il','il_util','open_rv_12m','open_rv_24m','max_bal_bc','all_util','total_rev_hi_lim',
               'inq_fi','total_cu_tl','inq_last_12m','acc_open_past_24mths','avg_cur_bal','bc_open_to_buy',
               'bc_util','chargeoff_within_12_mths','delinq_amnt','mo_sin_old_il_acct','mo_sin_old_rev_tl_op',
               'mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl','mort_acc','mths_since_recent_bc','mths_since_recent_bc_dlq',
               'mths_since_recent_inq','mths_since_recent_revol_delinq','num_accts_ever_120_pd','num_actv_bc_tl',
               'num_actv_rev_tl','num_bc_sats','num_bc_tl','num_il_tl','num_op_rev_tl','num_rev_accts',
               'num_rev_tl_bal_gt_0','num_sats','num_tl_120dpd_2m','num_tl_30dpd','num_tl_90g_dpd_24m',
               'num_tl_op_past_12m','pct_tl_nvr_dlq','percent_bc_gt_75','pub_rec_bankruptcies','tax_liens',
               'tot_hi_cred_lim','total_bal_ex_mort','total_bc_limit','total_il_high_credit_limit','next_pymnt_d',
              'last_credit_pull_d','initial_list_status','mths_since_last_delinq','mths_since_last_record','desc',
              'pymnt_plan','url'],axis=1)

In [ ]:
# some highlight from above cell, we are removing desc column bcoz of 15k approx missing data

In [39]:
#check shape of your dataframe i.e. how many rows and columns we have in total
df.shape

(39717, 41)

In [ ]:
# Even after removing many unwanted columns we can notice we still have 41 column, so lets remove some more unrelated or 
# dupllicate columns form the dataframe columns 

In [ ]:
df = df.drop(['id',''],axis=1)

In [ ]:
# we can remove "id" as this is duplicate with member_id both columns signify same things even though value sare differnnt

In [32]:
#get an overview of the dataframe
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-15,171.62
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-14,649.91
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-15,357.48
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,524.06,3513.330000,3513.33,2475.94,1037.39,0.00,0.00,0.00,May-16,67.79


## below are the analysis of the columns present in dataframe

In [13]:
#id
df["id"].nunique()
#unique id being 39717 which is equal to total number of rows - means its unique for each row 

39717

In [14]:
#member_id
df["member_id"].nunique()
# member id is same as our previous column "id" means we can discard on of these column in our later analysis

39717

In [20]:
#loan_amnt
df["loan_amnt"].value_counts()
#based on this distribution we can say multiple borrowers have applied for the same amount.
#And most request amount is 10k

10000    2833
12000    2334
5000     2051
6000     1908
15000    1895
         ... 
22875       1
8175        1
19475       1
21225       1
22550       1
Name: loan_amnt, Length: 885, dtype: int64

In [18]:
df["loan_amnt"].nunique()
# Total unique Loan applied for by the borrower are 885

885